In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "2H"
max_min = 45
min_min = max_min - 15
max_sec = max_min*60
min_sec = min_min*60
table_name = "final_in_poss_df"

In [3]:
print(f'{table_name}_{half}_{min_min}_{max_min}.csv')

final_in_poss_df_2H_30_45.csv


In [4]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchPeriod == half]
df2 = df2[df2.eventSec >= min_sec]
if max_min < 45:
    df2 = df2[df2.eventSec < max_sec]

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchPeriod == half]
df3 = df3[df3.eventSec >= min_sec]
if max_min < 45:
    df3 = df3[df3.eventSec < max_sec]


with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchPeriod == half]
df4 = df4[df4.eventSec >= min_sec]
if max_min < 45:
    df4 = df4[df4.eventSec < max_sec]


with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchPeriod == half]
df5 = df5[df5.eventSec >= min_sec]
if max_min < 45:
    df5 = df5[df5.eventSec < max_sec]


with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchPeriod == half]
df6 = df6[df6.eventSec >= min_sec]
if max_min < 45:
    df6 = df6[df6.eventSec < max_sec]


with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

In [16]:
all_events_df[all_events_df.eventSec > 60*45].eventSec.mean() / 60

47.09991180370034

In [15]:
all_events_df[all_events_df.eventSec > 60*45].eventSec.median() / 60

46.899080641666664

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [11]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [12]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.000000,0.030303,0.030303,0.939394
10531---2499736,0.076923,0.153846,0.145299,0.623932
10531---2499741,0.000000,0.043478,0.043478,0.913043
10531---2499757,0.015873,0.047619,0.047619,0.888889
10531---2499766,0.068966,0.068966,0.103448,0.758621
...,...,...,...,...
756---2565880,0.024096,0.096386,0.096386,0.783133
756---2565892,0.066667,0.120000,0.120000,0.693333
756---2565904,0.016129,0.048387,0.048387,0.887097


In [13]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [14]:
motif_stats

,mean,std
Motif,,
ABAB,0.039610,0.034200
ABCA,0.104820,0.037981
ABCB,0.105061,0.037967
ABCD,0.750509,0.084063


# Feature engineering - Shot DF

In [15]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-15-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [16]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-16-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [17]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-17-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [18]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [19]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499736,1,0,1,0
10531---2499757,3,0,1,2
10531---2499766,1,0,1,0
10531---2499796,1,1,0,0
10531---2499806,6,4,1,1
...,...,...,...,...
756---2565880,2,0,1,1
756---2565892,2,2,0,0
756---2565904,3,0,2,1


In [20]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [21]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499736,1,0.000000,1.000000,0.000000
10531---2499757,3,0.000000,0.333333,0.666667
10531---2499766,1,0.000000,1.000000,0.000000
10531---2499796,1,1.000000,0.000000,0.000000
10531---2499806,6,0.666667,0.166667,0.166667
...,...,...,...,...
756---2565880,2,0.000000,0.500000,0.500000
756---2565892,2,1.000000,0.000000,0.000000
756---2565904,3,0.000000,0.666667,0.333333


# Feature engineering - Pass DF - Pass Types

In [22]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-22-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [23]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [24]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
0,8,Simple pass,[{'id': 1802}],14869,"[{'y': 11, 'x': 17}, {'y': 5, 'x': 29}]",2499719,Pass,1609,2H,1801.863249,85,177960710,1609---2499719,1
2,8,Head pass,[{'id': 1801}],14869,"[{'y': 8, 'x': 17}, {'y': 10, 'x': 34}]",2499719,Pass,1609,2H,1803.124585,82,177960711,1609---2499719,1
3,8,Simple pass,[{'id': 1801}],167145,"[{'y': 10, 'x': 34}, {'y': 5, 'x': 42}]",2499719,Pass,1609,2H,1806.631558,85,177960714,1609---2499719,2
6,8,Simple pass,[{'id': 1801}],25413,"[{'y': 7, 'x': 36}, {'y': 17, 'x': 41}]",2499719,Pass,1609,2H,1807.877050,85,177960715,1609---2499719,2
14,8,Simple pass,[{'id': 1801}],192748,"[{'y': 85, 'x': 21}, {'y': 80, 'x': 17}]",2499719,Pass,1631,2H,1816.352145,85,177960708,1631---2499719,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570911,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1
570913,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2
570915,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2
570917,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3


In [25]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [26]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)
<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [27]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [28]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-28-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [29]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1802}],14869,"[{'y': 11, 'x': 17}, {'y': 5, 'x': 29}]",2499719,Pass,1609,2H,1801.863249,85,177960710,1609---2499719,1,15.178933,-0.321751,Forward Pass
2,8,Head pass,[{'id': 1801}],14869,"[{'y': 8, 'x': 17}, {'y': 10, 'x': 34}]",2499719,Pass,1609,2H,1803.124585,82,177960711,1609---2499719,1,20.462649,0.078271,Forward Pass
3,8,Simple pass,[{'id': 1801}],167145,"[{'y': 10, 'x': 34}, {'y': 5, 'x': 42}]",2499719,Pass,1609,2H,1806.631558,85,177960714,1609---2499719,2,10.400000,-0.394791,Forward Pass
6,8,Simple pass,[{'id': 1801}],25413,"[{'y': 7, 'x': 36}, {'y': 17, 'x': 41}]",2499719,Pass,1609,2H,1807.877050,85,177960715,1609---2499719,2,10.000000,0.927295,Side Pass
14,8,Simple pass,[{'id': 1801}],192748,"[{'y': 85, 'x': 21}, {'y': 80, 'x': 17}]",2499719,Pass,1631,2H,1816.352145,85,177960708,1631---2499719,1,6.248200,-2.446854,Backward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570911,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1,10.762899,-0.837981,Side Pass
570913,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2,18.000000,0.927295,Side Pass
570915,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2,32.179497,-0.608689,Forward Pass
570917,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3,7.211103,2.553590,Backward Pass


In [30]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [31]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1802}],14869,"[{'y': 11, 'x': 17}, {'y': 5, 'x': 29}]",2499719,Pass,1609,2H,1801.863249,85,177960710,1609---2499719,1,15.178933,-0.321751,Forward Pass
2,8,Head pass,[{'id': 1801}],14869,"[{'y': 8, 'x': 17}, {'y': 10, 'x': 34}]",2499719,Pass,1609,2H,1803.124585,82,177960711,1609---2499719,1,20.462649,0.078271,Forward Pass
3,8,Simple pass,[{'id': 1801}],167145,"[{'y': 10, 'x': 34}, {'y': 5, 'x': 42}]",2499719,Pass,1609,2H,1806.631558,85,177960714,1609---2499719,2,10.400000,-0.394791,Forward Pass
6,8,Simple pass,[{'id': 1801}],25413,"[{'y': 7, 'x': 36}, {'y': 17, 'x': 41}]",2499719,Pass,1609,2H,1807.877050,85,177960715,1609---2499719,2,10.000000,0.927295,Side Pass
14,8,Simple pass,[{'id': 1801}],192748,"[{'y': 85, 'x': 21}, {'y': 80, 'x': 17}]",2499719,Pass,1631,2H,1816.352145,85,177960708,1631---2499719,1,6.248200,-2.446854,Backward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570911,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1,10.762899,-0.837981,Side Pass
570913,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2,18.000000,0.927295,Side Pass
570915,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2,32.179497,-0.608689,Forward Pass
570917,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3,7.211103,2.553590,Backward Pass


In [32]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [33]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

<ipython-input-33-54e3dcb6a0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [34]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499726,0,6,1,7,3,1,8,6,4,7,11,18
10531---2499736,1,12,3,9,20,1,15,47,12,16,30,74
10531---2499741,1,2,6,9,10,6,2,7,6,9,25,15
10531---2499757,0,1,8,8,15,6,4,16,8,9,29,28
10531---2499766,0,3,1,6,10,3,2,2,5,4,19,9
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,1,8,3,10,19,5,6,23,11,12,34,40
756---2565892,1,4,3,11,11,8,5,17,18,8,30,40
756---2565904,4,4,3,14,9,0,10,7,14,11,23,31


## Pass Types

In [35]:
high_pass_types = ['High pass', 'Cross', 'Launch']

all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# --- overall ---
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# reindex to avoid missing columns (robust)
pass_counts['High_pass_Total'] = pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
pass_counts['Low_pass_Total']  = pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total']  = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

final_pass_types_df = pass_counts[['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']].copy()

# --- zone-specific ---
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_df = passes_df[passes_df['Zone'] == zone]

    zone_pass_counts = zone_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)

    zone_high_total = zone_pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
    zone_low_total  = zone_pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

    # avoid divide-by-zero: teams with 0 passes in that zone -> NaN
    denom = zone_pass_counts['Total_Passes'].replace(0, pd.NA)

    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_high_total / denom
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}']  = zone_low_total / denom


In [36]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3
uniqueTeamId,,,,,,,,,
10531---2499726,36,0.111111,0.861111,0.200000,0.733333,0.000000,1.000000,0.166667,0.833333
10531---2499736,120,0.041667,0.950000,0.080000,0.920000,0.037975,0.949367,0.000000,1.000000
10531---2499741,49,0.163265,0.836735,0.416667,0.583333,0.157895,0.842105,0.000000,1.000000
10531---2499757,66,0.257576,0.681818,0.416667,0.583333,0.156250,0.812500,0.318182,0.545455
10531---2499766,32,0.250000,0.718750,0.500000,0.500000,0.200000,0.733333,0.111111,0.888889
...,...,...,...,...,...,...,...,...,...
756---2565880,86,0.139535,0.848837,0.235294,0.764706,0.100000,0.900000,0.157895,0.789474
756---2565892,78,0.243590,0.756410,0.176471,0.823529,0.093750,0.906250,0.448276,0.551724
756---2565904,65,0.400000,0.584615,0.392857,0.607143,0.200000,0.800000,0.647059,0.294118


# Pass_Connectivity

In [37]:
from collections import defaultdict
import networkx as nx

In [38]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [39]:
test_dfdf = passes_df.iloc[:1000]

In [40]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [41]:
connectivity_results

[{'1609---2499719': 4.945454545454545, '1631---2499719': 2.08974358974359},
 {'1651---2499720': 1.0833333333333333, '1625---2499720': 6.654545454545454},
 {'1610---2499721': 4.777777777777778, '1646---2499721': 2.911111111111111},
 {'1673---2499722': 1.5818181818181818, '1628---2499722': 3.106060606060606},
 {'1623---2499723': 2.4909090909090907, '1639---2499723': 3.4727272727272727},
 {'1611---2499724': 4.358974358974359, '1633---2499724': 4.121212121212121},
 {'1624---2499725': 5.858974358974359, '1613---2499725': 3.088888888888889},
 {'1619---2499726': 4.242424242424242, '10531---2499726': 2.422222222222222},
 {'1612---2499727': 2.043956043956044, '1644---2499727': 3.9454545454545453},
 {'1659---2499728': 3.757575757575758, '1627---2499728': 2.6666666666666665},
 {'1644---2499729': 2.257575757575758, '1659---2499729': 3.3484848484848486},
 {'1646---2499730': 3.4363636363636365, '1627---2499730': 1.2777777777777777},
 {'1673---2499731': 2.090909090909091, '1613---2499731': 2.60606060

In [42]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [43]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1609---2499719,4.945455
1631---2499719,2.089744
1651---2499720,1.083333
1625---2499720,6.654545
1610---2499721,4.777778
...,...
3158---2576336,4.500000
3204---2576337,3.878788
3164---2576337,3.727273


In [44]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [45]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1,High_pass_Ratio_Zone2,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.000000,0.030303,0.030303,0.939394,36,0.111111,0.861111,0.200000,0.733333,0.000000,...,0.166667,0.111111,0.194444,0.305556,0.500000,NaN,NaN,NaN,NaN,2.422222
10531---2499736,0.076923,0.153846,0.145299,0.623932,120,0.041667,0.950000,0.080000,0.920000,0.037975,...,0.391667,0.100000,0.133333,0.250000,0.616667,1.0,0.000000,1.000000,0.000000,4.436364
10531---2499741,0.000000,0.043478,0.043478,0.913043,49,0.163265,0.836735,0.416667,0.583333,0.157895,...,0.142857,0.122449,0.183673,0.510204,0.306122,NaN,NaN,NaN,NaN,2.800000
10531---2499757,0.015873,0.047619,0.047619,0.888889,66,0.257576,0.681818,0.416667,0.583333,0.156250,...,0.242424,0.121212,0.136364,0.439394,0.424242,3.0,0.000000,0.333333,0.666667,3.015152
10531---2499766,0.068966,0.068966,0.103448,0.758621,32,0.250000,0.718750,0.500000,0.500000,0.200000,...,0.062500,0.156250,0.125000,0.593750,0.281250,1.0,0.000000,1.000000,0.000000,1.911111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.024096,0.096386,0.096386,0.783133,86,0.139535,0.848837,0.235294,0.764706,0.100000,...,0.267442,0.127907,0.139535,0.395349,0.465116,2.0,0.000000,0.500000,0.500000,4.054545
756---2565892,0.066667,0.120000,0.120000,0.693333,78,0.243590,0.756410,0.176471,0.823529,0.093750,...,0.217949,0.230769,0.102564,0.384615,0.512821,2.0,1.000000,0.000000,0.000000,3.890909
756---2565904,0.016129,0.048387,0.048387,0.887097,65,0.400000,0.584615,0.392857,0.607143,0.200000,...,0.107692,0.215385,0.169231,0.353846,0.476923,3.0,0.000000,0.666667,0.333333,3.696970


In [46]:
table_name = "final_in_poss_df"

if half is not None:
    final_df.to_csv(f'{table_name}_{half}_{min_min}_{max_min}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

In [47]:
half

'2H'

# END OF FILE